# Linearization and intersection in parameter space concept
<br>This example provides basic idea of how **````linearization````** and **````intersection````** steps are done with PSC.<br>
The structure parameter **coordinate** is defined to have four atomic coordinates *i.e.* the example uses <br>
four dimensional parameter space. To be simple the EPA framework with amplitude approach is used.<br><br>


1. [Linearization](#Linearization) cell calculates the magnitude of intensity for the given structure (<span style="color:red">***coordinate***</span>) <br>  and verifies the linearization os isosurface by invoking *checklinear* routine which uses the grid-based method 
2. [Finding intersection](#finding) cell take the polytope from [Linearization](#Linearization) cell and finds the common solution region between the polytope corresponding to different reflection **h**
3. [Writing data](#write) cell writes all simulated data from [Finding intersection](#finding) cell in a ***.h5*** formatted file
4. [Possible solution regions](#centroid) The exact polytope should contain the given coordinate. However, in [Finding intersection](#finding) routine, the PSC finding the exact solution region along with<br> all pseudo solutions for the given <span style="color:red">***coordinate***</span> <br><br>


<mark> User friendly and better examples are in Ex01 and Ex02 <mark>

## <font color = 'green'> **Importing modules**

In [1]:
%matplotlib widget

import warnings
warnings.filterwarnings('ignore')

import os
import time
import numpy as np
import polytope as pc


from psc.lib.g_space import g

from psc.lib.x3DlinearizationEPA import linearizenD_EPA
from psc.lib.x3Drepetition import getpolytope_EPA  
from psc.lib.x3Dchecklinearization import checklinear
from psc.lib.x3Dintersection import find_intersection
from psc.lib.x3Dreadwrite import wrtdata

from psc.lib.xlinearizationtools import radius_from_volume

## <font color = 'green'> **Example coordinate**

### Linearization

In [2]:
# ------------------------------------------------------------------------------------------------
# --->  Generate required information such as atomic coordinate to be solved
#       artificial atomic scattering factors. 'j' fixes the direction of third atomic coordinate
# ------------------------------------------------------------------------------------------------

coordinate = np.array([0.349, 0.362, 0.1615, 0.1615])
f    = [1.0]*len(coordinate)
j    = len(coordinate)-1


# ------------------------------------------------------------------------------------------------
# ---> Apply origin fixing rule. The origin is always fixed at [0, 0, ....]
# ------------------------------------------------------------------------------------------------
l = 1
coordinate = np.sort(coordinate)[::-1]  if (np.sign(g(l, coordinate, f))>0) else np.sort(0.5-coordinate)[::-1]

# ------------------------------------------------------------------------------------------------
# ---> Start to solve given atomic structure using first 4 number of reflections
# ------------------------------------------------------------------------------------------------

h  = 4
info, plist = [], []
IorG='intensity'
print("Assumed coordinate : ", coordinate)

for l in range(1,h+1):
        
    # ===> 1. initilization
    k  = 2*np.pi*l
    gi = np.abs(g(l, coordinate, f))
    amplitudesign = np.sign(g(l, coordinate, f))
    
    # ===> 2. linearization
    normal, distance, boundarypoints = linearizenD_EPA(l, f, gi)
    
    ST = time.time()
    # ===> 3. get all polytope
    p = getpolytope_EPA( l, normal, distance, amplitudesign, IorG, imax=0.5)
    plist.append(p)
    ET = time.time()
    print(f'===> Time taken for RO {l} is {ET-ST} sec.')
    info.append([l, normal, distance])
    
    # ===> 4. check linearization
    checklinear(l, f, gi, normal, distance, j=len(f)-1, n=50, s=1, testiso=True)
###


Assumed coordinate :  [0.3385 0.3385 0.151  0.138 ]
--> Polytope contains complete isosurface. Successful Linearization for RO = 1
===> Time taken for RO 1 is 0.04407334327697754 sec.


c:\Users\pearl\Desktop\2021_Freiberg\psc\lib\g_space.py:64: RuntimeWarning: invalid value encountered in arccos
  xj = (np.arccos(argm))/k


--> Polytope contains complete isosurface. Successful Linearization for RO = 1
--> Polytope contains complete isosurface. Successful Linearization for RO = 2
===> Time taken for RO 2 is 1.3480448722839355 sec.
--> Polytope contains complete isosurface. Successful Linearization for RO = 2
--> Polytope contains complete isosurface. Successful Linearization for RO = 3
===> Time taken for RO 3 is 2.0334668159484863 sec.
--> Polytope contains complete isosurface. Successful Linearization for RO = 3
isotype 1
===> Time taken for RO 4 is 6.48685622215271 sec.
--> Polytope contains complete isosurface. Successful Linearization for RO = 4


### <a id="finding"></a>Finding intersection

In [3]:
'''
Defining Asym and reduce no. of polytopes in the polytope list for each reflection order
condition: the polytope list must contain first order reflection info.

what if it is starts with other reflection order ?
'''

# Reducing the polytopes

temp = np.tril(np.ones(shape=(len(f), len(f))) , 0 )
temp = 0.5*np.vstack([[0]*len(normal), temp])
asym = pc.qhull(np.array(temp))

plistr=[]
for i in range(len(plist)):
    r = []
    for ij in plist[i]:
        if ij.intersect(asym):
            r.append(ij)
    plistr.append(r)
    print("===> RO is ",i+1," The len of polytope region before reduction : ",len(plist[i])," after reduction :", len(r), len(plistr))


# finding intersection

solution = []
for inx, ply in enumerate(plistr):
    print("\x1b[0;32m===> intersection for RO : %g"%(inx+1), end="   \x1b[0m")
    if inx==0:
        sf = pc.Region([asym.intersect(ply[0])]) # plistr[inx][0] = ply[0]
        solution.append(sf)
        print(f"len(solution) : {len(sf)} and container len {len(solution)}")
    else:
        tmp= find_intersection(solution[-1], pc.Region(ply))
        solution.append(tmp)
        print(f"len(solution) : {len(tmp)} and container len {len(solution)}")

print("\n===>\x1b[1;31m Variable \x1b[1;3;32msolution\x1b[0m\x1b[1;31m contains the intersection results")

===> RO is  1  The len of polytope region before reduction :  1  after reduction : 1 1
===> RO is  2  The len of polytope region before reduction :  32  after reduction : 10 2
===> RO is  3  The len of polytope region before reduction :  162  after reduction : 30 3
===> RO is  4  The len of polytope region before reduction :  512  after reduction : 70 4
===> intersection for RO : 1   len(solution) : 1 and container len 1
===> intersection for RO : 2   len(solution) : 5 and container len 2
===> intersection for RO : 3   len(solution) : 31 and container len 3
===> intersection for RO : 4   len(solution) : 19 and container len 4

===> Variable solution contains the intersection results


### <a id='write'></a> Writing data in h5 file

In [13]:
'''
This section creates hdf file and writes the available information. Current stand takes only one
solution. another for loop is to be added if all RO intersection information is to be written.
'''

# creating file and writing information ! <-- variable names should be regularized, not compatible with readwrite.py

fpath   = os.path.join(os.getcwd())
fn = os.path.join(fpath,'resultfile_%g.h5'%(h))

if os.path.isfile(fn):
    os.remove(fn)
    print(f"===> removed file {fn}")

total_solNr=len(solution[-1])
allsolutions=solution[-1]

for jj, i in enumerate(solution[-1]):
    if coordinate in i:
        rc = 0
        xg = np.mean(pc.extreme(i), axis=0)
        volume = pc.volume(i)
        
        extremepnts = pc.extreme(i)
        dmax = np.max(extremepnts, axis=0)
        dmin = np.min(extremepnts, axis=0)
        err  = np.abs(dmax-dmin)/2
        final = i
        
        volAsym = volume
        Lsol=len(solution[-1])
        grandradius=radius_from_volume(len(coordinate), volAsym)

localmat = pc.extreme(final) ; m = np.mean(localmat,0)
solution_error  = np.abs(np.max(localmat, axis=0) - np.min(localmat, axis=0))
solution_center = np.mean(localmat, 0)
solution_volume = pc.volume(final)

wrtdata(pair_inx=rc, fname=fn, solution=m, solution_polytope=final, solution_volume=solution_volume, solution_error=solution_error, solution_extremepnts=localmat, vol_in_Asym=volAsym, grandradius=grandradius , total_solNr=total_solNr, allsolutions=rc)

print(f"===> Data is written in {fn}")


===> removed file c:\Users\pearl\Desktop\2021_Freiberg\resultfile_4.h5
===> Data is written in c:\Users\pearl\Desktop\2021_Freiberg\resultfile_4.h5


### <a id='centroid'></a> Possible solutions and centroid of structure containing polytope

In [12]:
for jj, i in enumerate(solution[-1]):
    xg = np.mean(pc.extreme(i), axis=0)
    print(xg, jj, coordinate in i)
    if coordinate in i:
        extremepnts = pc.extreme(i)
        dmax = np.max(extremepnts, axis=0)
        dmin = np.min(extremepnts, axis=0)
        err  = np.abs(dmax-dmin)/1
        print(f"\x1b[1;2;32m--> polytope contain coordinate. Predicted \x1b[1;3;34m{xg}\x1b[0;2;32m and assumed \x1b[1;3;34m{coordinate}\x1b[0;2;32m. Error: \x1b[1;3;34m{err}\x1b[0m" )


[0.49977095 0.26651426 0.07529377 0.02509792] 0 False
[0.47495223 0.26958918 0.2332747  0.02939986] 1 False
[0.49255228 0.29390111 0.24255228 0.04390111] 2 False
[0.44436228 0.24842059 0.24526176 0.04932007] 3 False
[0.45718966 0.25479991 0.25159997 0.06214745] 4 False
[0.35627209 0.34124901 0.15875099 0.14372791] 5 True
--> polytope contain coordinate. Predicted [0.35627209 0.34124901 0.15875099 0.14372791] and assumed [0.3385 0.3385 0.151  0.138 ]. Error: [0.04061893 0.0298067  0.0298067  0.04061893]
[0.3658238  0.34778784 0.17810556 0.14046386] 6 False
[0.36135638 0.31858638 0.1506706  0.13374678] 7 False
[0.37907362 0.31573251 0.16154531 0.13455943] 8 False
[0.37943626 0.3127084  0.1557103  0.13762936] 9 False
[0.37921177 0.33809493 0.16284651 0.15520621] 10 False
[0.41435917 0.38811972 0.11188028 0.08564083] 11 False
[0.35828597 0.30725284 0.14583333 0.12090048] 12 False
[0.36559798 0.33797421 0.18490391 0.12086812] 13 False
[0.42958478 0.37078823 0.11240675 0.08722024] 14 False
[